Example script to generate text from Bible.
At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
text = open("/data/Bible.txt").read()
print(text[:1000])

﻿1:1 In the beginning God created the heavens and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the morning were the first day.

1:6 And God said, Let there be a firmament in the midst of the waters,
and let it divide the waters from the waters.

1:7 And God made the firmament, and divided the waters which were
under the firmament from the waters which were above the firmament:
and it was so.

1:8 And God called the firmament Heaven. And the evening and the
morning were the second day.

1:9 And God said, Let the waters under the heaven be gathered together
unto one place, and let the dry land appear: and it was so.

1:10 And God called the

In [3]:
print('corpus length:', len(text))

corpus length: 4332455


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


In [5]:
print(chars)

['\n', ' ', '!', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1444139


In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
#Build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               104448    
_________________________________________________________________
dense_1 (Dense)              (None, 75)                9675      
_________________________________________________________________
activation_1 (Activation)    (None, 75)                0         
Total params: 114,123
Trainable params: 114,123
Non-trainable params: 0
_________________________________________________________________


In [9]:
def sample(preds, temperature=1.0):
    """
    helper function to sample an index from a probability array
    """ 
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [10]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
1444139/1444139 [==============================] - 435s 301us/step - loss: 1.8325

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "wn cause: remember how the foolish
man r"
wn cause: remember how the foolish
man rear to the LORD of the earth the LORD hath the LORD hath he said the LORD said unto the LORD said unto the LORD said unto the LORD sall the son of the LORD hath may shall be for the son of the LORD of the LORD had server to the LORD hath said unto the LORD hath for the son of the LORD man the LORD said unto the LORD hath the LORD said unto the LORD hath the LORD hath hear and the sent of the LORD 
----- diversity: 0.5
----- Generating with seed: "wn cause: remember how the foolish
man r"
wn cause: remember how the foolish
man ry mine of the people, and the sent of Israel.

1:9 And the LORD said unto resmine and the son of the gether, that were and hear and the right congelled to hear to prouse of the
people of the hing of the s

put all that all the flechet is pull faith: they pertitude
out?  bepilied with years of the priest against nor
pitser's burdeth, fillided, and thise bring
shew: and I tall oney nom the deast that was
woman; O LORD hath nught.

16:2 And this sould set aboth me, show rest that I have against them, and they shall purned to he
her take the mouth to da
----- diversity: 1.2
----- Generating with seed: "the house: 14:36 Then the priest shall c"
the house: 14:36 Then the priest shall came to polf you a man was
me on Juddeh,
and a pertimies shall he sf putath also alk laub: 3:5 And Put thou hard in a wink hath not efter, every sond
hombers weith Israel, and whou he beproarts; whon
been the poackenss, nor veringed, whom Darion thus
salveth shalt te pars fearth to past before it, Vitish forty uncles alsy whoseevers that are a lest of you
places unto unto thine Henders us, say will
Epoch 5/60
1444139/1444139 [==============================] - 431s 298us/step - loss: 1.2592

----- Generating text a

40:35 And Moses was not able to enter into the house of the LORD, and the sons of Judah, and the son of Jerohoah, and Abiah, which the LORD hath see the house of the LORD, and the sons of Judah, and the sons of Johah, and Benaiah, and Abiah, and Jerusalem, and the sons of Jerobah, and the sons of Jerobah, and Abiah, and the sons of Judah, and Abiah, and the men of Judah, and the priests of the LORD, and the sin offerings of the head of 
----- diversity: 0.5
----- Generating with seed: "40:35 And Moses was not able to enter in"
40:35 And Moses was not able to enter into the wilderness, and the house of the LORD, and the
mount of the children of Israel, and will of the coverath of the son of the LORD.

10:19 And they have come to put provide of singment.

14:1 And he said unto thee, And he said, Now when they
seest thou make a congregation that the LORD have a son of
Egypt to him to him.

26:2 And Joseph said unto them, What my mare and servants and the land of
----- diversity: 1.0
-----

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


, and the children of Israel, and said unto them
----- diversity: 0.5
----- Generating with seed: "34:5 They looked unto him, and were ligh"
34:5 They looked unto him, and were lighted in the land of Israel.

23:18 And they had beared to him against thee, and the side of the world and men to the congregation,
and the mount of the souns, and the children of Israel.

23:10 And the sin offering in the trutt of the LORD, and in the prophets,
that the presence of the sons of Judah, and all the fire that shall be an old things like their
lift of the house of the LORD the LORD of h
----- diversity: 1.0
----- Generating with seed: "34:5 They looked unto him, and were ligh"
34:5 They looked unto him, and were light as Calomot, they
said, Sy set the seventh people a great spoved for the skin of
beast.

20:35 In thine height of the prophet are rad, and in the valley of an
every inothers, and never, I skin in the day.

42:11 Moraim with him a ass, and takest that we shall east I decaured thee, so


...

3:10 But, because he could eat of all thirteenarifuly: 1:9 So that
did he opened Mordecas in flowed.

39:15 And it cause them, and set were fulfilded unto
----- diversity: 1.2
----- Generating with seed: "unto the heads
of the fathers of the tri"
unto the heads
of the fathers of the tribring, every one;
32:14 Peacsiant of the covenant of his table please, a
God of heigh: but they were: they buriging.

39:5 Theng hath ye thut will bring?  4:30 A centing Jacob Balaal's depart is Edom, and died.

10:16 Then the blimnce speaked that which Katheah; went done, Which ray.

1:8 Also shalt thou we nothind his nighte gate.

5:8  Talk of it: that David was rones of people me:
filled and wa
Epoch 44/60
1444139/1444139 [==============================] - 430s 298us/step - loss: 1.0660

----- Generating text after Epoch: 43
----- diversity: 0.2
----- Generating with seed: "9:8 Then the nations set against him on "
9:8 Then the nations set against him on the sea of the children of Israel, and the prie

God, and shall be a perfect and the men of Judah.

33:8 And he said, Art thou to the sign, and said, I will sing all thy treasure; and the man answered
him, and said, I will not answer the house
of the LORD: and he shall come by the sanctuary, and the LORD will she shall be
a son.

15:14 And when they also answered and said, He made thee, and his word shall be the streets, and the man was come to prea
----- diversity: 1.0
----- Generating with seed: " let the king remember the LORD thy
God,"
 let the king remember the LORD thy
God, thou shalt not a God by the signethous of my judges, and the daughters, and shall thereof they die, he fell upon
thy land: behold, thery stone was Bethal, and brought a queenure on my newak Siza, the LORD commanded Agno of all the voice of Asas, before thee, and all the east sheep; and they had a me which
was effed by
far in the year the LORD; to short upon them shall lay the inhabitants of the r
----- diversity: 1.2
----- Generating with seed: " let the kin

given us against my sinner: for the famine; 26:31 But no jisen leave: The king shall die, ye arms abuten it mocks, which he
mountus unto his own
Epoch 51/60
1444139/1444139 [==============================] - 430s 298us/step - loss: 1.0604

----- Generating text after Epoch: 50
----- diversity: 0.2
----- Generating with seed: "d he said, Is not he rightly named Jacob"
d he said, Is not he rightly named Jacob.

1:16 And the LORD said unto them, I am the LORD, and the LORD hath spoken in the sword.

2:10 And the LORD said unto them, The LORD hath spoken the same of the sea, and the children of Israel that said, I am the LORD, and the son of the first of the children of Israel, and the sons of Jacob, and Jacob, and the children of Judah, and the sons of Jacob, and the sons of Jehorak, and the children o
----- diversity: 0.5
----- Generating with seed: "d he said, Is not he rightly named Jacob"
d he said, Is not he rightly named Jacob.

14:1 And the LORD said unto them, Behold, the LORD is 

:41 All that pass by the way spoil him: but thou shalt be clean,
and they and fishes of their houses, not the great thirty thousand children.

14:18 And he sent up to Saraon, soith with the words of Judah, and
was chometse whethout the people remembered, unto me
previned together, and the fowls of the quainting purple is a very mouth, and say
unto them adul, Neither lots, and revelesting
him, and so mempence old from him out of the two 
----- diversity: 1.2
----- Generating with seed: ":41 All that pass by the way spoil him: "
:41 All that pass by the way spoil him: they sent him
mod from the LORD Groon, and how the people tree avise, and ender begat
Paun, and the young king of Ramad.

1:15 And their nakeders thereof he devouckion.

49:11 The voidest day, Gebarter and I wanes to see that as one are not
vexaters, O Jeare
given; and sticketh them; and the pleasuch
shall be a mightinus, whither I am have charged
unto them whither hear heared; say f ary
with him,
Epoch 55/60
1444139/144413

12:14 Then shall the seas of the LORD was the sons of Jehoiakim, and the sons of Jehoah the son of Jehoahazzar the son of Jehoahaz the son of Jehoahaz the son of Jehoahaz the son of Jehoahazzar, and Mattan, and Jeremiah, and Maacha, and Jacob, the son of Jehoahaziah, and Jehoiada, and the sons of Aaron, and the sons of Jehoahaziah, and Jehoi
----- diversity: 0.5
----- Generating with seed: "me the sun be hot,
ye shall have help. A"
me the sun be hot,
ye shall have help. And when they said, Behold, thou shalt make them into his
sins.

26:1 And when he said unto him, Hear, I pray thee, and have
not shall be come out and be the sacrifices of the LORD for ever came unto the
house of God: but the house was they had destroyed the house of Judah his father with him the wait in the city, that he went to the mount of the LORD her winds.

2:25 And the children of Judah said
----- diversity: 1.0
----- Generating with seed: "me the sun be hot,
ye shall have help. A"
me the sun be hot,
ye shall hav

In [12]:
model.save("data/bible_character_rnn.h5")

In [39]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [40]:
!nvidia-smi

Sun Apr 29 08:51:20 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 387.34                 Driver Version: 387.34                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro K620         Off  | 00000000:01:00.0 Off |                  N/A |
| 34%   41C    P0     2W /  30W |   1804MiB /  2000MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  